# LASSO Test Results Analysis - 2 Stocks (SAP, ASML)

This notebook analyzes the test results from running the LASSO pipeline on 2 stocks.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import Counter

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

%matplotlib inline

## 1. Load Test Results

In [ ]:
# Load training metadata
metadata_path = '../data/processed/models/with_us_stocks/lasso_test/test_training_metadata.csv'
metadata = pd.read_csv(metadata_path)
metadata['model_date'] = pd.to_datetime(metadata['model_date'])

# Load baseline comparison
baseline_path = '../data/processed/models/with_us_stocks/lasso_test/baseline_comparison_corrected.csv'
baseline_comp = pd.read_csv(baseline_path)

print(f"Total models trained: {len(metadata)}")
print(f"Unique tickers: {metadata['ticker'].nunique()}")
print(f"Date range: {metadata['model_date'].min().strftime('%Y-%m')} to {metadata['model_date'].max().strftime('%Y-%m')}")

metadata.head()

## 2. Baseline Comparison

Compare LASSO model (index + Russell 1000) vs Baseline (index only)

In [ ]:
print("BASELINE vs LASSO PERFORMANCE")
print("=" * 70)
print(f"\n{'Metric':<25} {'Baseline':<15} {'LASSO':<15} {'Improvement':<15}")
print("-" * 70)
print(f"{'Mean IC':<25} {baseline_comp['baseline_ic'].mean():>14.4f} {baseline_comp['lasso_ic'].mean():>14.4f} {baseline_comp['ic_improvement'].mean():>14.4f}")
print(f"{'Median IC':<25} {baseline_comp['baseline_ic'].median():>14.4f} {baseline_comp['lasso_ic'].median():>14.4f} {baseline_comp['ic_improvement'].median():>14.4f}")

rel_improvement = (baseline_comp['lasso_ic'].mean() / baseline_comp['baseline_ic'].mean() - 1) * 100
print(f"\nRelative improvement: {rel_improvement:.1f}%")
print(f"LASSO better in {(baseline_comp['ic_improvement'] > 0).mean() * 100:.1f}% of models")

In [ ]:
# Visualize baseline comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# IC comparison
ax1 = axes[0]
x = np.arange(len(baseline_comp))
width = 0.35
ax1.bar(x - width/2, baseline_comp['baseline_ic'], width, label='Baseline (Index Only)', alpha=0.8)
ax1.bar(x + width/2, baseline_comp['lasso_ic'], width, label='LASSO (Index + Russell)', alpha=0.8)
ax1.set_xlabel('Model')
ax1.set_ylabel('IC')
ax1.set_title('IC Comparison: Baseline vs LASSO', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# IC improvement
ax2 = axes[1]
colors = ['green' if x > 0 else 'red' for x in baseline_comp['ic_improvement']]
ax2.bar(range(len(baseline_comp)), baseline_comp['ic_improvement'], color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(0, color='black', linewidth=1, linestyle='--')
ax2.set_xlabel('Model')
ax2.set_ylabel('IC Improvement')
ax2.set_title('IC Improvement (LASSO - Baseline)', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Scatter: baseline vs lasso
ax3 = axes[2]
for ticker in baseline_comp['ticker'].unique():
    ticker_data = baseline_comp[baseline_comp['ticker'] == ticker]
    ax3.scatter(ticker_data['baseline_ic'], ticker_data['lasso_ic'], 
               label=ticker, s=80, alpha=0.7, edgecolors='black', linewidth=0.5)
ax3.plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5, label='No Improvement')
ax3.set_xlabel('Baseline IC')
ax3.set_ylabel('LASSO IC')
ax3.set_title('Baseline vs LASSO IC', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✓ Points above diagonal = LASSO better than baseline")
print(f"✓ {(baseline_comp['ic_improvement'] > 0).sum()} out of {len(baseline_comp)} models improved")

## 3. Overall IC Statistics

In [ ]:
print("IC Distribution Statistics (on residuals):")
print(metadata['val_ic'].describe())

print(f"\n% of models with IC > 0.00: {(metadata['val_ic'] > 0.00).mean() * 100:.1f}%")
print(f"% of models with IC > 0.10: {(metadata['val_ic'] > 0.10).mean() * 100:.1f}%")
print(f"% of models with IC > 0.20: {(metadata['val_ic'] > 0.20).mean() * 100:.1f}%")
print(f"% of models with IC > 0.50: {(metadata['val_ic'] > 0.50).mean() * 100:.1f}%")

In [ ]:
# IC Distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.hist(metadata['val_ic'], bins=20, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(0, color='red', linestyle='--', linewidth=2, label='IC = 0')
ax1.axvline(metadata['val_ic'].median(), color='green', linestyle='--', linewidth=2, 
           label=f'Median = {metadata["val_ic"].median():.3f}')
ax1.set_xlabel('Validation IC (on residuals)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of IC Across All Models', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
stock_ic = metadata.groupby('ticker')['val_ic'].mean().sort_values(ascending=True)
colors = ['green' if x > 0.3 else 'steelblue' if x > 0.15 else 'orange' for x in stock_ic.values]
ax2.barh(range(len(stock_ic)), stock_ic.values, color=colors, edgecolor='black')
ax2.set_yticks(range(len(stock_ic)))
ax2.set_yticklabels(stock_ic.index)
ax2.set_xlabel('Average IC')
ax2.set_title('Average IC by Stock', fontweight='bold')
ax2.axvline(0.15, color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax2.axvline(0.3, color='gray', linestyle=':', linewidth=1, alpha=0.5)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 4. Performance Over Time

In [ ]:
# IC time series by stock
fig, ax = plt.subplots(figsize=(14, 6))

for ticker in sorted(metadata['ticker'].unique()):
    ticker_data = metadata[metadata['ticker'] == ticker].sort_values('model_date')
    ax.plot(ticker_data['model_date'], ticker_data['val_ic'], 
           marker='o', label=ticker, linewidth=2, markersize=8)

ax.axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Validation IC', fontsize=12)
ax.set_title('IC Time Series by Stock', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Model Characteristics

In [ ]:
print("Model Sparsity:")
print(f"  Mean non-zero coefs: {metadata['n_nonzero_coefs'].mean():.1f} out of 996")
print(f"  Median: {metadata['n_nonzero_coefs'].median():.0f}")
print(f"  % of features used: {metadata['n_nonzero_coefs'].mean() / 996 * 100:.1f}%")

print(f"\nRegularization Parameter:")
print(f"  Mean alpha: {metadata['alpha'].mean():.6f}")
print(f"  Median alpha: {metadata['alpha'].median():.6f}")
print(f"  Range: [{metadata['alpha'].min():.6f}, {metadata['alpha'].max():.6f}]")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sparsity distribution
ax1 = axes[0]
ax1.hist(metadata['n_nonzero_coefs'], bins=20, edgecolor='black', alpha=0.7, color='mediumseagreen')
ax1.axvline(metadata['n_nonzero_coefs'].mean(), color='red', linestyle='--', linewidth=2,
           label=f'Mean = {metadata["n_nonzero_coefs"].mean():.0f}')
ax1.set_xlabel('Number of Non-Zero Coefficients')
ax1.set_ylabel('Frequency')
ax1.set_title('Model Sparsity (out of 996 features)', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Alpha vs IC
ax2 = axes[1]
for ticker in metadata['ticker'].unique():
    ticker_data = metadata[metadata['ticker'] == ticker]
    ax2.scatter(np.log10(ticker_data['alpha']), ticker_data['val_ic'], 
               label=ticker, s=60, alpha=0.7, edgecolors='black', linewidth=0.5)
ax2.set_xlabel('log10(Alpha)')
ax2.set_ylabel('Validation IC')
ax2.set_title('IC vs Regularization Strength', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(0, color='red', linestyle='--', linewidth=1, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Feature Importance

In [ ]:
# Extract top features
all_top_features = []
for features_str in metadata['top_5_features'].dropna():
    features = features_str.split(',')
    all_top_features.extend(features)

feature_counts = Counter(all_top_features)
top_features_df = pd.DataFrame(feature_counts.most_common(20), columns=['Feature', 'Count'])
top_features_df['Feature'] = top_features_df['Feature'].str.replace('russell_', '')

print("Top 20 Most Frequently Selected Russell Stocks:")
for i, row in top_features_df.iterrows():
    pct = row['Count'] / len(metadata) * 100
    print(f"  {i+1:2d}. {row['Feature']:6s} - selected in {row['Count']:2d} models ({pct:.1f}%)")

In [ ]:
# Visualize top features
fig, ax = plt.subplots(figsize=(10, 8))
ax.barh(range(len(top_features_df)), top_features_df['Count'], edgecolor='black', alpha=0.7)
ax.set_yticks(range(len(top_features_df)))
ax.set_yticklabels(top_features_df['Feature'])
ax.set_xlabel('Selection Count', fontsize=11)
ax.set_title('Most Frequently Selected Russell 1000 Stocks (Top 20)', fontsize=12, fontweight='bold')
ax.invert_yaxis()
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

## 7. Summary Statistics by Stock

In [ ]:
summary = metadata.groupby('ticker').agg({
    'val_ic': ['mean', 'std', 'min', 'max', 'count'],
    'alpha': 'mean',
    'n_nonzero_coefs': 'mean'
}).round(4)

summary.columns = ['IC_mean', 'IC_std', 'IC_min', 'IC_max', 'n_models', 'alpha_mean', 'n_features_mean']
summary = summary.sort_values('IC_mean', ascending=False)

print("\nSummary by Stock:")
print(summary)

## 8. Key Takeaways

In [ ]:
print("=" * 70)
print("KEY TAKEAWAYS")
print("=" * 70)

avg_baseline = baseline_comp['baseline_ic'].mean()
avg_lasso = baseline_comp['lasso_ic'].mean()
avg_improvement = baseline_comp['ic_improvement'].mean()
rel_improvement = (avg_lasso / avg_baseline - 1) * 100

print(f"\n1. BASELINE PERFORMANCE (Index Beta Only)")
print(f"   • IC = {avg_baseline:.3f}")
print(f"   • Index beta already captures substantial information")

print(f"\n2. LASSO PERFORMANCE (Index Beta + Russell 1000)")
print(f"   • IC = {avg_lasso:.3f}")
print(f"   • Improvement: +{avg_improvement:.3f} IC points")
print(f"   • Relative gain: +{rel_improvement:.1f}%")

print(f"\n3. CONSISTENCY")
pct_better = (baseline_comp['ic_improvement'] > 0).mean() * 100
print(f"   • LASSO outperforms in {pct_better:.0f}% of models")
print(f"   • {(baseline_comp['ic_improvement'] > 0.05).mean() * 100:.0f}% have improvement > 0.05")

print(f"\n4. MODEL CHARACTERISTICS")
print(f"   • Average {metadata['n_nonzero_coefs'].mean():.0f} features selected (out of 996)")
print(f"   • LASSO effectively performing feature selection")
print(f"   • Most important: KLAC, AGNC, JNJ, AOS")

print(f"\n5. CONCLUSION")
print(f"   ✓ Russell 1000 adds meaningful value beyond index beta")
print(f"   ✓ {rel_improvement:.0f}% improvement in predictive accuracy")
print(f"   ✓ Approach validated - ready for full deployment")

print("\n" + "=" * 70)